In [11]:
import pandas as pd

df_monthly = pd.read_csv('/Users/vietanh/Documents/doan/preprocess/monthly_data.csv')
df_monthly['Date'] = pd.to_datetime(df_monthly['Date'])
df_monthly['year-month'] = df_monthly['Date'].dt.to_period('M').astype(str)

print(df_monthly.head())


  Ticker       Date   Open   High    Low  Close  Volume       VWAP year-month
0    A32 2018-10-31  12.85  12.85  12.85  12.85       0   0.000000    2018-10
1    A32 2018-11-30  12.85  15.88  12.85  15.88    1000  14.870000    2018-11
2    A32 2018-12-31  15.88  15.88  13.90  15.84    2200  15.101458    2018-12
3    A32 2019-01-31  16.78  16.78  16.78  17.60     200  15.216275    2019-01
4    A32 2019-02-28  16.78  16.78  12.10  12.10     300  15.090090    2019-02


In [12]:
# Fix version of the user's code with corrected index alignment for rolling calculations
df_monthly = pd.read_csv('/Users/vietanh/Documents/doan/preprocess/monthly_data.csv')

# Convert 'Date' to datetime and extract 'year-month'
df_monthly['Date'] = pd.to_datetime(df_monthly['Date'])
df_monthly['year-month'] = df_monthly['Date'].dt.to_period('M').astype(str)

# Group by Ticker and year-month to calculate monthly OHLCV
monthly_grouped = df_monthly.groupby(['Ticker', 'year-month']).agg({
    'Open': 'first',
    'High': 'max',
    'Low': 'min',
    'Close': 'last',
    'Volume': 'sum'
}).reset_index()

# Add AdjClose as same as Close (if no adjustment)
monthly_grouped['AdjClose'] = monthly_grouped['Close']

# Sort data
monthly_grouped = monthly_grouped.sort_values(by=['Ticker', 'year-month'])

# Calculate metrics
monthly_grouped['ret'] = monthly_grouped.groupby('Ticker')['Close'].pct_change()
monthly_grouped['Volatility'] = monthly_grouped.groupby('Ticker')['ret'].rolling(3).std().reset_index(level=0, drop=True)
monthly_grouped['HL'] = monthly_grouped['High'] - monthly_grouped['Low']
monthly_grouped['LO'] = monthly_grouped['Close'] - monthly_grouped['Low']
monthly_grouped['SR'] = monthly_grouped['Close'] - monthly_grouped['Open']
monthly_grouped['PM'] = (monthly_grouped['High'] + monthly_grouped['Low']) / 2
monthly_grouped['MDD'] = monthly_grouped.groupby('Ticker')['Close'].apply(lambda x: (x / x.cummax()) - 1).reset_index(level=0, drop=True)
monthly_grouped['TVV'] = monthly_grouped.groupby('Ticker')['Volume'].rolling(3).std().reset_index(level=0, drop=True)

# Compute ATR
prev_close = monthly_grouped.groupby('Ticker')['Close'].shift(1)
tr = pd.concat([
    monthly_grouped['High'] - monthly_grouped['Low'],
    (monthly_grouped['High'] - prev_close).abs(),
    (monthly_grouped['Low'] - prev_close).abs()
], axis=1).max(axis=1)
monthly_grouped['ATR'] = tr.groupby(monthly_grouped['Ticker']).rolling(3).mean().reset_index(level=0, drop=True)

# Median HL
monthly_grouped['Median_HL'] = (monthly_grouped['High'] + monthly_grouped['Low']) / 2

# Forward return
monthly_grouped['fore_di_rt'] = monthly_grouped.groupby('Ticker')['Close'].shift(-1) / monthly_grouped['Close'] - 1
monthly_grouped['variation_t'] = monthly_grouped.groupby('Ticker')['Close'].pct_change()

# Moving averages and std
monthly_grouped['ma7_t'] = monthly_grouped.groupby('Ticker')['Close'].transform(lambda x: x.rolling(7).mean())
monthly_grouped['ma14_t'] = monthly_grouped.groupby('Ticker')['Close'].transform(lambda x: x.rolling(14).mean())
monthly_grouped['ma21_t'] = monthly_grouped.groupby('Ticker')['Close'].transform(lambda x: x.rolling(21).mean())
monthly_grouped['s_d7_t'] = monthly_grouped.groupby('Ticker')['Close'].transform(lambda x: x.rolling(7).std())

# Risk-free rate and excess return
monthly_grouped['rf'] = 0.005  # fixed monthly rf
monthly_grouped['RET'] = monthly_grouped['ret'] - monthly_grouped['rf']

# Placeholder for missing columns
for col in ['SK','KU','RSI','rsq','mvol','mlo','rvni','rsqvni','mlovni','mvolvni','Mkt']:
    monthly_grouped[col] = np.nan

# Final column order
required_columns = [
    'year-month','Ticker','Open','High','Low','Close','AdjClose','Volume',
    'ret','Volatility','HL','LO','SR','PM','MDD','TVV','ATR','SK','KU','RSI',
    'Median_HL','fore_di_rt','variation_t','ma7_t','ma14_t','ma21_t','s_d7_t',
    'rf','RET','rsq','mvol','mlo','rvni','rsqvni','mlovni','mvolvni','Mkt'
]
monthly_final = monthly_grouped[required_columns]

# Save final result
monthly_result_path = "/Users/vietanh/Documents/doan/preprocess/monthly_data_final.csv"
monthly_final.to_csv(monthly_result_path, index=False)




In [19]:
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis
from sklearn.linear_model import LinearRegression

# Load dữ liệu cổ phiếu
df_monthly = pd.read_csv('/Users/vietanh/Documents/doan/preprocess/monthly_data.csv')
df_monthly['Date'] = pd.to_datetime(df_monthly['Date'])
df_monthly['year-month'] = df_monthly['Date'].dt.to_period('M').astype(str)

# Load dữ liệu VNI (chung thị trường)
df_vni = pd.read_csv('/Users/vietanh/Documents/doan/preprocess/monthly_vni.csv')
df_vni['Date'] = pd.to_datetime(df_vni['Date'])
df_vni['year-month'] = df_vni['Date'].dt.to_period('M').astype(str)
df_vni = df_vni.groupby('year-month').agg({
    'Close': 'last'
}).rename(columns={'Close': 'VNI_Close'})
df_vni['Mkt'] = df_vni['VNI_Close'].pct_change()
df_vni = df_vni.reset_index()

# Group dữ liệu cổ phiếu theo tháng
monthly_grouped = df_monthly.groupby(['Ticker', 'year-month']).agg({
    'Open': 'first',
    'High': 'max',
    'Low': 'min',
    'Close': 'last',
    'Volume': 'sum'
}).reset_index()
monthly_grouped['AdjClose'] = monthly_grouped['Close']

# Merge VNI vào
monthly_grouped = monthly_grouped.merge(df_vni[['year-month', 'Mkt']], on='year-month', how='left')

# Tính các chỉ số
monthly_grouped = monthly_grouped.sort_values(by=['Ticker', 'year-month'])
monthly_grouped['ret'] = monthly_grouped.groupby('Ticker')['Close'].pct_change()
monthly_grouped['Volatility'] = monthly_grouped.groupby('Ticker')['ret'].rolling(3).std().reset_index(level=0, drop=True)
monthly_grouped['HL'] = monthly_grouped['High'] - monthly_grouped['Low']
monthly_grouped['LO'] = monthly_grouped['Close'] - monthly_grouped['Low']
monthly_grouped['SR'] = monthly_grouped['Close'] - monthly_grouped['Open']
monthly_grouped['PM'] = (monthly_grouped['High'] + monthly_grouped['Low']) / 2
monthly_grouped['MDD'] = monthly_grouped.groupby('Ticker')['Close'].apply(lambda x: (x / x.cummax()) - 1).reset_index(level=0, drop=True)
monthly_grouped['TVV'] = monthly_grouped.groupby('Ticker')['Volume'].rolling(3).std().reset_index(level=0, drop=True)

# Tính ATR
prev_close = monthly_grouped.groupby('Ticker')['Close'].shift(1)
tr = pd.concat([
    monthly_grouped['High'] - monthly_grouped['Low'],
    (monthly_grouped['High'] - prev_close).abs(),
    (monthly_grouped['Low'] - prev_close).abs()
], axis=1).max(axis=1)
monthly_grouped['ATR'] = tr.groupby(monthly_grouped['Ticker']).rolling(3).mean().reset_index(level=0, drop=True)

# Tính Median_HL
monthly_grouped['Median_HL'] = (monthly_grouped['High'] + monthly_grouped['Low']) / 2

# Tính forward return và variation
monthly_grouped['fore_di_rt'] = monthly_grouped.groupby('Ticker')['Close'].shift(-1) / monthly_grouped['Close'] - 1
monthly_grouped['variation_t'] = monthly_grouped.groupby('Ticker')['Close'].pct_change()

# MA & Std
monthly_grouped['ma7_t'] = monthly_grouped.groupby('Ticker')['Close'].transform(lambda x: x.rolling(7).mean())
monthly_grouped['ma14_t'] = monthly_grouped.groupby('Ticker')['Close'].transform(lambda x: x.rolling(14).mean())
monthly_grouped['ma21_t'] = monthly_grouped.groupby('Ticker')['Close'].transform(lambda x: x.rolling(21).mean())
monthly_grouped['s_d7_t'] = monthly_grouped.groupby('Ticker')['Close'].transform(lambda x: x.rolling(7).std())

# Skew & Kurtosis
monthly_grouped['SK'] = monthly_grouped.groupby('Ticker')['ret'].transform(lambda x: x.rolling(3).apply(skew, raw=True))
monthly_grouped['KU'] = monthly_grouped.groupby('Ticker')['ret'].transform(lambda x: x.rolling(3).apply(kurtosis, raw=True))

# RSI
def calculate_rsi(series, window=14):
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window).mean()
    rs = gain / (loss + 1e-6)
    return 100 - (100 / (1 + rs))

monthly_grouped['RSI'] = monthly_grouped.groupby('Ticker')['Close'].transform(lambda x: calculate_rsi(x, 14))

# Rf, RET
monthly_grouped['rf'] = 0.005
monthly_grouped['RET'] = monthly_grouped['ret'] - monthly_grouped['rf']

# mvol, mlo, rvni
monthly_grouped['mvol'] = monthly_grouped.groupby('Ticker')['Volume'].transform(lambda x: x.rolling(3).mean())
monthly_grouped['mlo'] = monthly_grouped.groupby('Ticker')['LO'].transform(lambda x: x.rolling(3).mean())
monthly_grouped['rvni'] = monthly_grouped.groupby('Ticker')['ret'].transform(lambda x: x.rolling(3).std())

# rsq, rsqvni, mlovni, mvolvni
def calc_rsq(x, y):
    if len(x) < 3 or len(y) < 3:
        return np.nan
    model = LinearRegression()
    x = np.array(x).reshape(-1, 1)
    y = np.array(y)
    model.fit(x, y)
    return model.score(x, y)

monthly_grouped['rsq'] = monthly_grouped.groupby('Ticker').apply(
    lambda g: g['ret'].rolling(6).apply(lambda x: calc_rsq(np.arange(len(x)), x), raw=False)
).reset_index(level=0, drop=True)

monthly_grouped['rsqvni'] = monthly_grouped.groupby('Ticker').apply(
    lambda g: g[['Mkt', 'ret']].rolling(6).apply(lambda x: calc_rsq(x[:, 0], x[:, 1]) if not x.isnull().values.any() else np.nan, raw=False)
).reset_index(level=0, drop=True)

monthly_grouped['mlovni'] = monthly_grouped.groupby('Ticker').apply(
    lambda g: g[['Mkt', 'LO']].rolling(3).mean().reset_index(level=0, drop=True)['LO']
).reset_index(level=0, drop=True)

monthly_grouped['mvolvni'] = monthly_grouped.groupby('Ticker').apply(
    lambda g: g[['Mkt', 'Volume']].rolling(3).mean().reset_index(level=0, drop=True)['Volume']
).reset_index(level=0, drop=True)

# Xuất file kết quả
required_columns = [
    'year-month','Ticker','Open','High','Low','Close','AdjClose','Volume',
    'ret','Volatility','HL','LO','SR','PM','MDD','TVV','ATR','SK','KU','RSI',
    'Median_HL','fore_di_rt','variation_t','ma7_t','ma14_t','ma21_t','s_d7_t',
    'rf','RET','rsq','mvol','mlo','rvni','rsqvni','mlovni','mvolvni','Mkt'
]

monthly_final = monthly_grouped[required_columns]
monthly_final.to_csv("/Users/vietanh/Documents/doan/preprocess/monthly_data_final_full.csv", index=False)


FileNotFoundError: [Errno 2] No such file or directory: '/Users/vietanh/Documents/doan/preprocess/monthly_vni.csv'

In [13]:
monthly_final['year-month'] = pd.to_datetime(monthly_final['year-month'])

start_month = pd.to_datetime("2013-01")
end_month = pd.to_datetime("2025-03")

filtered_df = monthly_final[
    (monthly_final['year-month'] >= start_month) &
    (monthly_final['year-month'] <= end_month)
]

tickers_start = filtered_df[filtered_df['year-month'] == start_month]['Ticker'].unique()
tickers_end = filtered_df[filtered_df['year-month'] == end_month]['Ticker'].unique()
tickers_valid = set(tickers_start).intersection(set(tickers_end))

final_filtered_df = filtered_df[filtered_df['Ticker'].isin(tickers_valid)]

filtered_output_path = "/Users/vietanh/Documents/doan/preprocess/monthly_data_input.csv"
final_filtered_df.to_csv(filtered_output_path, index=False)




In [14]:
import pandas as pd

file_path = "/Users/vietanh/Documents/doan/preprocess/bubble.csv"
df = pd.read_csv(file_path)

# Đếm số lượng mã duy nhất
unique_tickers = df['Ticker'].nunique()

# Hiển thị danh sách mã (nếu cần)
tickers_list = df['Ticker'].unique()

print(f"Số lượng mã cổ phiếu: {unique_tickers}")
print("Danh sách các mã:", tickers_list)


Số lượng mã cổ phiếu: 721
Danh sách các mã: ['AAA' 'AAM' 'ABI' 'ABT' 'ACB' 'ACC' 'ACE' 'ACL' 'ADC' 'ADP' 'AGF' 'AGM'
 'AGR' 'ALT' 'ALV' 'AMC' 'AME' 'AMV' 'ANV' 'APC' 'APG' 'API' 'APP' 'APS'
 'ARM' 'ASA' 'ASM' 'ASP' 'ATA' 'AVF' 'BBC' 'BBS' 'BCC' 'BCE' 'BDB' 'BED'
 'BHC' 'BHP' 'BIC' 'BKC' 'BLF' 'BMC' 'BMI' 'BMJ' 'BMP' 'BPC' 'BRC' 'BSC'
 'BSI' 'BST' 'BT6' 'BTG' 'BTH' 'BTP' 'BTS' 'BTT' 'BTW' 'BVG' 'BVH' 'BVN'
 'BVS' 'BWA' 'BXH' 'C21' 'C32' 'C47' 'C92' 'CAD' 'CAN' 'CAP' 'CCI' 'CCL'
 'CCM' 'CDC' 'CHP' 'CI5' 'CID' 'CIG' 'CII' 'CJC' 'CKV' 'CLC' 'CLG' 'CLW'
 'CMC' 'CMG' 'CMI' 'CMS' 'CMT' 'CMV' 'CMX' 'CNG' 'CNT' 'COM' 'CPC' 'CSC'
 'CSM' 'CT3' 'CT6' 'CTA' 'CTB' 'CTD' 'CTG' 'CTI' 'CTN' 'CTS' 'CTX' 'CVN'
 'CVT' 'CX8' 'CYC' 'D11' 'D2D' 'DAC' 'DAD' 'DAE' 'DAG' 'DBC' 'DBM' 'DBT'
 'DC2' 'DC4' 'DCL' 'DCS' 'DCT' 'DDM' 'DDN' 'DGT' 'DHA' 'DHC' 'DHG' 'DHM'
 'DHT' 'DIC' 'DID' 'DIG' 'DIH' 'DL1' 'DLD' 'DLG' 'DLR' 'DMC' 'DNC' 'DNL'
 'DNM' 'DNP' 'DNT' 'DPC' 'DPM' 'DPP' 'DPR' 'DQC' 'DRC' 'DRH' 'DRL' 'DSN'
 'DST' 

In [15]:
from tqdm import tqdm
from statsmodels.tsa.stattools import adfuller

monthly_df = pd.read_csv('/Users/vietanh/Documents/doan/preprocess/monthly_data_input.csv')
monthly_df['year-month'] = pd.to_datetime(monthly_df['year-month'])

tickers = monthly_df['Ticker'].unique()
results = []
df_all_bubbles = pd.DataFrame()

# Loop over each ticker
for ticker in tqdm(tickers):
    df_ticker = monthly_df[monthly_df['Ticker'] == ticker].copy()

    # Filter invalid or too short
    df_ticker = df_ticker[df_ticker['AdjClose'] > 0]
    if len(df_ticker) < 25:
        results.append({'Ticker': ticker, 'bubble_percent': np.nan})
        continue

    # Calculate log price
    df_ticker['log_price'] = np.log(df_ticker['AdjClose'])
    df_ticker['is_bubble'] = 0

    total_periods = len(df_ticker)
    min_window = 20

    # Rolling ADF test for bubble labeling
    for start in range(0, total_periods - min_window):
        for end in range(start + min_window, total_periods):
            window_data = df_ticker['log_price'].iloc[start:end]

            if window_data.isna().any() or not np.isfinite(window_data).all():
                continue

            try:
                adf_stat, p_value, *_ = adfuller(window_data, maxlag=1, regression='c', autolag=None)
                if p_value < 0.05 and adf_stat < -2.9:
                    df_ticker.loc[df_ticker.index[end], 'is_bubble'] = 1
            except:
                continue

    bubble_percent = round(df_ticker['is_bubble'].sum() / len(df_ticker) * 100, 2)
    results.append({'Ticker': ticker, 'bubble_percent': bubble_percent})

    df_all_bubbles = pd.concat([df_all_bubbles, df_ticker], ignore_index=True)

labeled_path = "/Users/vietanh/Documents/doan/preprocess/bubble.csv"
summary_path = "/Users/vietanh/Documents/doan/preprocess/bubble_summary_percent.csv"
df_all_bubbles.to_csv(labeled_path, index=False)
pd.DataFrame(results).to_csv(summary_path, index=False)



  0%|          | 0/721 [00:00<?, ?it/s]

100%|██████████| 721/721 [28:13<00:00,  2.35s/it]   


In [16]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

# Load file đầy đủ có cột log_price và is_bubble
file_path = "/Users/vietanh/Documents/doan/preprocess/bubble.csv"
df = pd.read_csv(file_path)

# Danh sách các biến đầu vào
significant_vars = [
    'Open', 'High', 'Low', 'Volume', 'ret', 'HL', 'MDD', 'fore_di_rt',
    'rf', 'mvol', 'mlo', 'mlovni', 'mvolvni', 'Optimal_K', 'log_price'
]

# Lọc các biến có sẵn
available_vars = [col for col in significant_vars if col in df.columns]

# Chuẩn bị cột y_pred mặc định = NaN
df['y_pred'] = np.nan

# Lấy từng ticker và gán nhãn dự đoán vào cột mới
tickers = df['Ticker'].unique()

for ticker in tickers:
    df_ticker = df[df['Ticker'] == ticker].copy()

    # Xử lý missing và outliers
    df_ticker = df_ticker.replace([np.inf, -np.inf], np.nan)
    df_ticker = df_ticker.dropna(subset=available_vars + ['is_bubble'])

    if len(df_ticker) < 100:
        continue

    # Tạo X, y
    X = df_ticker[available_vars]
    y = df_ticker['is_bubble']

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    X_train = sm.add_constant(X_train)
    X_test = sm.add_constant(X_test)

    try:
        logit_model = sm.Logit(y_train, X_train).fit(disp=False)
        y_pred_prob = logit_model.predict(X_test)
        y_pred = (y_pred_prob > 0.5).astype(int)

        # Gán nhãn ngược lại vào DataFrame gốc theo index
        df.loc[X_test.index, 'y_pred'] = y_pred

    except Exception as e:
        print(f"[{ticker}] Error: {str(e)}")
        continue

# Sau khi xong, in preview và lưu file
df['y_pred'] = df['y_pred'].astype('Int64')  # giữ định dạng có thể chứa NaN
print(df[['year-month', 'Ticker', 'log_price', 'is_bubble', 'y_pred']].dropna().head())

# Lưu kết quả ra file
df.to_csv("/Users/vietanh/Documents/doan/preprocess/bubble_with_pred.csv", index=False)


Empty DataFrame
Columns: [year-month, Ticker, log_price, is_bubble, y_pred]
Index: []


In [17]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Load lại dữ liệu đã gán nhãn
df = pd.read_csv("/Users/vietanh/Documents/doan/preprocess/bubble.csv")

# Các cột không dùng cho hồi quy
excluded_cols = ['year-month', 'Ticker', 'is_bubble']

# Lấy danh sách cột đầu vào
feature_cols = [col for col in df.columns if col not in excluded_cols]

# Tính tỷ lệ NaN
na_ratio = df[feature_cols].isna().mean()

# Chỉ giữ lại các cột có 0% NaN
safe_cols = na_ratio[na_ratio == 0].index.tolist()

# Tạo X và y
X = df[safe_cols].copy()
X = X.replace([np.inf, -np.inf], np.nan).dropna()  # loại inf
y = df.loc[X.index, 'is_bubble']  # lấy y tương ứng với X

# Thêm hệ số chặn
X = sm.add_constant(X)

# Chạy OLS
ols_model = sm.OLS(y, X).fit()

# Trả về kết quả hồi quy
ols_model.summary().as_text()


'                            OLS Regression Results                            \n==============================================================================\nDep. Variable:              is_bubble   R-squared:                       0.027\nModel:                            OLS   Adj. R-squared:                  0.027\nMethod:                 Least Squares   F-statistic:                     368.6\nDate:                Mon, 21 Apr 2025   Prob (F-statistic):               0.00\nTime:                        23:51:34   Log-Likelihood:                -74885.\nNo. Observations:              105275   AIC:                         1.498e+05\nDf Residuals:                  105266   BIC:                         1.499e+05\nDf Model:                           8                                         \nCovariance Type:            nonrobust                                         \n==============================================================================\n                 coef    std err       

In [18]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

# Load dữ liệu đã gán nhãn bubble
file_path = "/Users/vietanh/Documents/doan/preprocess/bubble.csv"
df = pd.read_csv(file_path)

# Các biến có p-value < 0.05 theo kết quả OLS
significant_vars = [
    'High', 'Low', 'Close', 'AdjClose', 'HL', 'LO', 'PM',
    'MDD', 'Median_HL', 'fore_di_rt', 'rf', 'log_price'
]

# Xử lý lỗi nếu thiếu biến
available_vars = [var for var in significant_vars if var in df.columns]

# Lưu kết quả
logit_results = []

# Chọn 10 mã đầu tiên để demo
sample_tickers = df['Ticker'].unique()[:10]

for ticker in sample_tickers:
    df_ticker = df[df['Ticker'] == ticker].copy()

    # Xử lý missing và vô hạn
    df_ticker = df_ticker.replace([np.inf, -np.inf], np.nan)
    df_ticker = df_ticker.dropna(subset=available_vars + ['is_bubble'])

    if len(df_ticker) < 100:
        continue

    X = df_ticker[available_vars]
    y = df_ticker['is_bubble']

    # Chia dữ liệu
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # Thêm constant cho statsmodels
    X_train = sm.add_constant(X_train)
    X_test = sm.add_constant(X_test)

    try:
        # Huấn luyện mô hình
        logit_model = sm.Logit(y_train, X_train).fit(disp=False)
        y_pred_prob = logit_model.predict(X_test)
        y_pred = (y_pred_prob > 0.5).astype(int)

        acc = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logit_results.append({
            'Ticker': ticker,
            'Accuracy (%)': round(acc * 100, 2),
            'F1-score': round(f1, 4)
        })

    except Exception as e:
        logit_results.append({
            'Ticker': ticker,
            'Accuracy (%)': 'Error',
            'F1-score': '-',
            'Error': str(e)
        })

# In kết quả
logit_df = pd.DataFrame(logit_results)
print(logit_df)

# Lưu kết quả nếu cần
logit_df.to_csv("/Users/vietanh/Documents/doan/logit_results_summary.csv", index=False)


  Ticker Accuracy (%) F1-score            Error
0    AAA        Error        -  Singular matrix
1    AAM        Error        -  Singular matrix
2    ABI        Error        -  Singular matrix
3    ABT         80.0   0.8333              NaN
4    ACB        Error        -  Singular matrix
5    ACC        Error        -  Singular matrix
6    ACE        Error        -  Singular matrix
7    ACL         70.0   0.7097              NaN
8    ADC        Error        -  Singular matrix
9    ADP        Error        -  Singular matrix


/opt/anaconda3/envs/code/lib/python3.9/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/opt/anaconda3/envs/code/lib/python3.9/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
